In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [1]:
sample_sub = pd.read_csv('sample_submission.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Users\leosimoe\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\leosimoe\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
train_x = train.drop(['id','Class'],axis=1)
train_y = train['Class']

In [9]:
!pip install imbalanced-learn

  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)
  Using cached scikit_learn-1.2.1-cp39-cp39-win_amd64.whl (8.4 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfully uninstalled scikit-learn-0.24.2
  Using cached imbalanced_learn-0.10.1-py3-none-any.whl (226 kB)
  Using cached scikit_learn-1.2.1-cp39-cp39-win_amd64.whl (8.4 MB)
  Using cached joblib-1.2.0-py3-none-any.whl (297 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.24.2
    Uninstalling scikit-learn-0.24.2:
      Successfull

In [10]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

In [11]:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(train_x, train_y)

In [17]:
test_x = test.drop(['id'], axis=1)

In [20]:
!pip install lightgbm

In [21]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix

In [22]:
sc = StandardScaler()
sc.fit(X_resampled)
X = sc.transform(X_resampled)
X_test = sc.transform(test_x)

In [23]:
def score(clf, x, y):
    return roc_auc_score(y, clf.predict_proba(x)[:,1])

In [52]:
preds = [] # saving the preds of test set for each model 
scores = []
for train_ind, test_ind in rsk.split(X,y_resampled):
    x_train, x_test = X[train_ind], X[test_ind]
    y_train, y_test = y_resampled[train_ind], y_resampled[test_ind]
    xgb_params = {'n_estimators': 150,
                 'max_depth': 3,
                 'learning_rate': 0.1,
                 'min_child_weight': 4,
                 'subsample': 0.7,
                 'colsample_bytree': 0.3
            }
    #xgb = XGBClassifier(max_depth=2,reg_lambda=0.1)
    xgb = XGBClassifier(**xgb_params)
    xgb.fit(x_train, y_train)
    preds.append(xgb.predict_proba(X_test)[:,1])
    print(str(score(xgb,x_train, y_train)) + "    " + str(score(xgb,x_test, y_test)))
    scores.append(score(xgb,x_test, y_test))

0.9661930668584481    0.9652932451819888
0.9661930668584481    0.9652932451819888
0.9661464270585689    0.9652923594247155
0.9661464270585689    0.9652923594247155
0.9658222711918657    0.9656901345061845
0.9658222711918657    0.9656901345061845
0.9669544932698828    0.9665390078844236
0.9669544932698828    0.9665390078844236
0.9642111040941608    0.9642341053670416
0.9642111040941608    0.9642341053670416
0.9655445791407048    0.9642570963630877
0.9655445791407048    0.9642570963630877
0.9660606459589023    0.9672710982969375
0.9660606459589023    0.9672710982969375
0.9649117463083952    0.9639765936622245
0.9649117463083952    0.9639765936622245
0.9661159355741994    0.9664024004787598
0.9661159355741994    0.9664024004787598
0.9654446409784223    0.964787260261271
0.9654446409784223    0.964787260261271


In [53]:
predictions = np.mean(preds,axis=0)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X, y_resampled, test_size=0.2, random_state=42)

In [57]:
X_test1 = sc.transform(test_x)

In [58]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_prob_rf = rf.predict_proba(X_test)

# XGBoost
xgb1 = XGBClassifier()
xgb1.fit(X_train, y_train)
y_prob_xgb = xgb1.predict_proba(X_test)

# LightGBM
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train)
y_prob_lgbm = lgbm.predict_proba(X_test)

In [59]:
y_prob_rf_t = rf.predict_proba(X_test1)
y_prob_xgb_t = xgb.predict_proba(X_test1)
y_prob_lgbm_t = lgbm.predict_proba(X_test1)

In [60]:
prob_rf_t = y_prob_rf_t[:,1]
prob_lf_t = y_prob_lgbm_t[:,1]
prob_xf_t = y_prob_xgb_t[:,1]

(146087,)

(146087,)

In [82]:
test_sub = pd.DataFrame(test.id, columns=['id'])
test_sub['id'] = test.id
test_sub['Class'] = prob_rf_t*0.25+prob_lf_t*0.25+prob_xf_t*0.25+predictions*0.25
test_sub

,id,Class
0,219129,0.116662
1,219130,0.142484
2,219131,0.037661
3,219132,0.217966
4,219133,0.044376
...,...,...
146082,365211,0.056573
146083,365212,0.055353
146084,365213,0.138877
146085,365214,0.033937


,id,Class
0,219129,0.116662
1,219130,0.142484
2,219131,0.037661
3,219132,0.217966
4,219133,0.044376
...,...,...
146082,365211,0.056573
146083,365212,0.055353
146084,365213,0.138877
146085,365214,0.033937


In [83]:
test_sub.to_csv('test_sub_xgb_random_lgbm7111.csv', index=False)

In [ ]:
y_pred=clf.predict(X_test)

In [65]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [68]:
rf_test=RandomForestClassifier()

In [73]:
grid_param={
    'n_estimators': [100,200],
    'criterion':['gini', 'entropy'],
    #'max_depth': range(1,11,1),
    'min_samples_leaf': range(1,10,2),
    'min_samples_split':range(1,10,2),
    'max_features':['auto','log2']
    
}

In [74]:
gs=GridSearchCV(rf_test, param_grid=grid_param, cv=5, n_jobs=-1,verbose=3)

In [75]:
gs.fit(X_train,y_train)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


KeyboardInterrupt: 

KeyboardInterrupt: 